# FEATURE ENGINEERING: Military bases

In [2]:
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
scrubbed_data = pd.read_csv('../raw_data/scrubbed.csv', low_memory=False)

In [13]:
uscities_data = pd.read_csv('../raw_data/uscities.csv')

In [14]:
military_data = uscities_data[uscities_data['military'] == True ]

In [18]:
# military_data.shape
uscities_data.shape

(28338, 17)